In [2]:
# !pip install fake_useragent

### Selenium으로 크롤링

In [1]:
# !pip install selenium

In [1]:
from selenium import webdriver

In [3]:
url= 'http://www.zdnet.co.kr/newskey/?lstcode=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'

In [4]:
driver =  webdriver.Chrome()
driver.get(url)

In [5]:
#여러데이터 모으기
articles = driver.find_elements_by_css_selector("body > div.contentWrapper > div > div.left_cont > section > div")
len(articles)

17

In [6]:
#제목만 뽑아내기
import pandas as pd

title_list = []
for article in articles:
    title = article.find_element_by_css_selector("div.assetText > a > h3").text 
    title_list.append(title)
title =pd.DataFrame(title_list, columns=['title'])
title.head()

,title
0,"아마존 AI 알렉사, 람보르기니에서도 사용한다"
1,"“5G·AI에 집중”…정부, 핵심기술 R&D에 7천862억 투입"
2,"의료 AI 기업 루닛, 300억원 투자 유치"
3,中 최초 고속도로 트럭 L4 자율주행 테스트
4,6일부터 美 AI 소프트웨어 수출 제한...민감한 중국


In [7]:
#링크만 뽑아내기

link_list = []
for article in articles:
    link = article.find_element_by_css_selector("a").get_attribute("href")
    link_list.append(link)
link= pd.DataFrame(link_list, columns=['link'])
link.head() #상위 5개 보여줌

,link
0,http://www.zdnet.co.kr/view/?no=20200107070507
1,http://www.zdnet.co.kr/view/?no=20200106170838
2,http://www.zdnet.co.kr/view/?no=20200106134518
3,http://www.zdnet.co.kr/view/?no=20200106090544
4,http://www.zdnet.co.kr/view/?no=20200106080955


In [12]:
#상세내용 뽑아내기
preview_list = []
for article in articles:
    preview = article.find_element_by_css_selector("div.assetText > a > p").text
    preview_list.append(preview)
preview=pd.DataFrame(preview_list,columns=['preview'])
preview.head()

,preview
0,아마존 인공지능(AI) 비서인 알렉사를 람보르기니 차량과 리비안 차량에서 사용할 수...
1,정부가 올해 5G 인공지능(AI) 경쟁력을 강화하기 위한 핵심 기술 연구·개발(R&...
2,의료 인공지능(AI) 대표기업 루닛(대표 서범석)이 국내외 7개 기관 투자자로부터 ...
3,중국에서 인공지능(AI) 자율주행 트럭으로 택배를 배송하는 미래가 한발짝 더 가까워...
4,미국 정부가 중국으로 수출되는 인공지능(AI) 소프트웨어를 제한하는 정책을 6일 발...


In [13]:
result=pd.concat([title, link,preview], axis=1)
result.tail() #하위5개 보여줌

,title,link,preview
12,AI 시대 이용자보호 정책 전담조직 생긴다,http://www.zdnet.co.kr/view/?no=20200102111519,인공지능(AI) 시대 이용자 보호 정책의 전담조직이 마련됐다. 방송통신위원회는 2일...
13,"산업부, 디지털통상 전문인력 양성한다",http://www.zdnet.co.kr/view/?no=20200102110525,산업통상자원부는 서울대 국제대학원(원장 정종호)과 손잡고 '디지털통상 아카데미' 시...
14,"구글 딥마인드 ""의사보다 유방암 오판율 낮은 AI모델 개발""",http://www.zdnet.co.kr/view/?no=20200102101522,구글 인공지능(AI) 전문 자회사 딥마인드가 유방암 검진 오판율을 줄이는 성과를 거...
15,마이너리티 리포트가 현실로…'AI+CCTV'로 범죄 예측 가능해진다,http://www.zdnet.co.kr/view/?no=20200102091441,국내 연구진이 과거 범죄 통계정보와 CCTV 영상을 자동 분석해 범죄 등 위험 상황...
16,"뤼이드, 1억 건 넘는 AI 학습 데이터 공개",http://www.zdnet.co.kr/view/?no=20200102091438,AI 튜터 솔루션 기업 뤼이드(대표 장영준)가 교육 분야 AI 학습 데이터베이스 '...


In [2]:
# 함수로 만들기
def get_articles(page):
    driver =  webdriver.Chrome()
    url = 'http://www.zdnet.co.kr/newskey/?lstcode=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&page={}'.format(page)
    driver.get(url)
    #여러데이터 모으기
    articles = driver.find_elements_by_css_selector("body > div.contentWrapper > div > div.left_cont > section > div")
    
    #제목
    title_list = []
    for article in articles:
        title = article.find_element_by_css_selector("div.assetText > a > h3").text 
        title_list.append(title)
    title =pd.DataFrame(title_list, columns=['title'])
    
    #링크만 뽑아내기
    link_list = []
    for article in articles:
        link = article.find_element_by_css_selector("a").get_attribute("href")
        link_list.append(link)
    link= pd.DataFrame(link_list, columns=['link'])
    
    #상세내용 뽑아내기
    preview_list = []
    for article in articles:
        preview = article.find_element_by_css_selector("div.assetText > a > p").text
        preview_list.append(preview)
    preview=pd.DataFrame(preview_list,columns=['preview'])
    
    result=pd.concat([title, link, preview], axis=1)
    driver.quit()
    return result

In [3]:
# 자동으로 일정 페이지 크롤링해줘서 데이터프레임 만들어주기
def zd_net(startpage, endpage):
    import pandas as pd
    df=pd.DataFrame([])
    for i in range(startpage, endpage+1):
        temp=get_articles(i)
        df= pd.concat([df,temp],axis=0)
    #index 번호 다시 매기기
    df.reset_index(drop=True, inplace=True)
    return df
      

In [4]:
final_df= zd_net(1,4)

In [5]:
final_df.tail(3)

,title,link,preview
74,"구글, AI가 선별한 유익한 기사 상단에 먼저 표출",http://www.zdnet.co.kr/view/?no=20191212095140,구글이 뉴스의 홍수 속에서 보다 풍부한 내용을 담고 있는 뉴스만 골라 검색 결과 상...
75,"삼성전기, AI 기술로 '1천억원' 아낀다",http://www.zdnet.co.kr/view/?no=20191212091448,삼성전기가 인공지능 기술을 통해 1천41억원에 달하는 비용을 절감하는 성과를 창출했...
76,"내년 ICT 부처 예산, AI·소부장 중심으로 확대",http://www.zdnet.co.kr/view/?no=20191211150537,10일 국회 본회의를 통과한 내년도 예산안 기금운용계획 수정안에서 ICT 산업을 담...


In [6]:
final_df.to_csv("zd_net.csv", index=False, encoding='utf-8' )

In [22]:
## 참고
### csv파일을 여기서 읽고 싶다면
import pandas as pd
final_df = pd.read_csv("zd_net.csv", encoding="utf-8")


### 만약 엑셀에서 열고 싶다면
1. 엑셀 열기
2. 데이터 탭 > 외부데이터 가져오기> 아무거나 누르기> 모든파일> csv 파일선택
3. 언어를 utf-8로 설정하고 열기